In [1]:
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.features
import rasterio.warp
from rasterio.plot import show
from matplotlib import pyplot
from bs4 import BeautifulSoup
import requests
import time
import json

In [ ]:
# import pyogrio; pyogrio.list_drivers()

# import fiona; fiona.supported_drivers

In [2]:
# map = gpd.read_file(
#     "/media/muskrat/060A9B8E0A9B78FF/eco_data/feow/feow-map.zip!GIS_hs_snapped"
# )
# map = gpd.read_file("/media/muskrat/060A9B8E0A9B78FF/eco_data/soil_maps/wss_gsmsoil_US_[2016-10-13]/wss_gsmsoil_US_[2016-10-13]/spatial/gsmsoilmu_a_us.shp")
# map = gpd.read_file(
#     "/media/muskrat/060A9B8E0A9B78FF/eco_data/soil_maps/wss_SSA_OH009_soildb_OH_2003_[2023-08-31]/OH009/spatial/soilmu_a_oh009.shp"
# )
# map = rasterio.open(
#     "/media/muskrat/060A9B8E0A9B78FF/eco_data/soil_maps/HWSD2_RASTER/HWSD2.bil"
# )

map = gpd.read_file("/media/muskrat/T7 Shield/eco_data/v4/maps/final/DMSW.geojson")

In [ ]:
map

In [4]:
map.crs

In [ ]:
# plot map with bounding box

# show((map, 1), cmap="viridis")

# pyplot.imshow(map.read(1))


# pyplot.show()

In [ ]:
# with rasterio.open(
#     "/media/muskrat/060A9B8E0A9B78FF/eco_data/soil_maps/HWSD2_RASTER/HWSD2.bil"
# ) as dataset:

#     # read dataset metadata
#     # print(dataset.profile)

#     show(dataset)

In [3]:
with rasterio.open(
    "/media/muskrat/060A9B8E0A9B78FF/eco_data/soil_maps/HWSD2_RASTER/HWSD2.bil"
) as dataset:

    # Read the dataset's valid data mask as a ndarray.
    mask = dataset.dataset_mask()

    # map equals empty geodataframe
#     finish_dict = {"type": "FeatureCollection", "features": []}

#     # Extract feature shapes and values from the array.
#     for geom, val in rasterio.features.shapes(mask, transform=dataset.transform):

#         # Transform shapes from the dataset's own coordinate
#         # reference system to CRS84 (EPSG:4326).
#         geom = rasterio.warp.transform_geom(dataset.crs, "EPSG:4326", geom, precision=6)

#         # Create a GeoDataFrame from the shapes and values
#         # and add it to the map
#         # map = map.append(gpd.GeoDataFrame({'value': val}, geometry=[geom], index=[val]))
#         # print(geom)
#         init_dict = {"type": "Feature", "properties": {"value": val}, "geometry": geom}

#         finish_dict["features"].append(init_dict)

# # write to geojson
# with open("/media/muskrat/T7 Shield/eco_data/v3/hwsd2/HWSD2.geojson", "w") as f:
#     json.dump(finish_dict, f)

In [ ]:
# find unique values in mask

# unique = list(set(mask.flatten()))

# unique

In [ ]:
map.plot()

In [ ]:
# plot map with x and y limits
xmin, ymin, xmax, ymax = (-82.1, 39.3, -82, 39.5)

# base = map.plot(color="white", edgecolor="black")
ax = map.plot()
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
# ax.invert_xaxis()

In [ ]:
# GET FEOW IDS

# for i in range of 0 to 29 including 29
trs = []
for i in range(1, 30, 1):
    time.sleep(10)
    url = f"https://www.feow.org/ecoregions/list?page={i}"

    print(url)

    page = requests.get(url)

    soup = BeautifulSoup(page.content, "html.parser")

    tr = soup.find_all("tr")

    trs.extend(tr)

trs

In [ ]:
# remove value at trs[0]

trs.pop(0)

trs

In [ ]:
# for each tr in trs remove tr that contains th tag with data-field="id"

for tr in trs:
    if tr.find("th", {"data-field": "id"}):
        trs.remove(tr)

trs

In [ ]:
# create dataframe with columns id and name

df = pd.DataFrame(columns=["id", "name"])

for tr in trs:
    # print(tr.find("td").text)
    id = tr.find("td").text

    # print(tr.find('a').text)
    name = tr.find("a").text
    # remove \n from name
    name = name.replace("\n", "")

    df = df.append({"id": id, "name": name}, ignore_index=True)

df

In [ ]:
# rename column feow_id to id in map

map.rename(columns={"FEOW_ID": "id"}, inplace=True)

map

In [ ]:
# remove area_skm column from map

map.drop(columns=["AREA_SKM"], inplace=True)

map

In [ ]:
# convert df id column to int

df["id"] = df["id"].astype("int")

df

In [ ]:
# merge df and map on id

merge = pd.merge(df, map, on="id")

merge

In [ ]:
# convert merge to geodataframe

merge = gpd.GeoDataFrame(merge, geometry="geometry")

merge

In [ ]:
# set crs to epsg:4326

merge = merge.set_crs(epsg=4326)

merge

In [ ]:
merge.crs

In [ ]:
# write merge to geojson

merge.to_file(
    "/media/muskrat/T7 Shield/eco_data/v3/feow/feow.geojson", driver="GeoJSON"
)

In [ ]:
# HWSD

map = gpd.read_file("/media/muskrat/T7 Shield/eco_data/v3/hwsd2/HWSD2.geojson")

map

In [5]:
map.crs

In [ ]:
map.geometry

In [ ]:
# create slice of map that contains first ten rows of map

slice = map[47753:47754]

slice

In [ ]:
slice.plot()

In [4]:
# remove value column from map

map = map.drop(columns=["value"])

slice = slice.drop(columns=["value"])

# map

In [ ]:
slice.crs

In [ ]:
# map.plot(alpha=0.5, edgecolor="k")
# map.boundary.plot()


# xmin, ymin, xmax, ymax = (-150, 60, -50, 84)

base = map.plot(color="white", edgecolor="black")

ax = slice.plot(ax=base, cmap="viridis")
# ax.set_xlim(xmin, xmax)
# ax.set_ylim(ymin, ymax)

In [ ]:
# DSMW

# remove columns COUNTRY, SQKM, CNTNAME, CNTCODE, PERMAFROST,MISCLU1, MISCLU2, PHASE1, PHASE2,

map = map.drop(
    columns=[
        "COUNTRY",
        "SQKM",
        "CNTNAME",
        "CNTCODE",
        "PERMAFROST",
        "MISCLU1",
        "MISCLU2",
        "PHASE1",
        "PHASE2",
    ]
)

map

In [ ]:
# print unique values of domsoi

map["DOMSOI"].unique()

In [ ]:
specific_soil_name = {
    "A": "ACRISOLS",
    "Ao": "Orthic Acrisols",
    "Af": "Ferric Acrisols",
    "Ah": "Humic Acrisols",
    "Ap": "Plinthic Acrisols",
    "Ag": "Gleyic Acrisols",
    "B": "CAMBISOLS",
    "Be": "Eutric Cambisols",
    "Bd": "Dystric Cambisols",
    "Bh": "Humic Cambisols",
    "Bg": "Gleyic Cambisols",
    "Bx": "Gelic Cambisols",
    "Bk": "Calcic Cambisols",
    "Bc": "Chromic Cambisols",
    "Bv": "Vertic Cambisols",
    "Bf": "Ferralic Cambisols",
    "C": "CHERNOZEMS",
    "Ch": "Haplic Chernozems",
    "Ck": "Calcic Chernozems",
    "Cl": "Luvic Chernozems",
    "Cg": "Glossic Chernozems",
    "D": "PODZOLUVISOLS",
    "De": "Eutric Podzoluvisols",
    "Dd": "Dystric Podzoluvisols",
    "Dg": "Gleyic Podzoluvisols",
    "E": "RENDZINAS",
    "F": "FERRALSOLS",
    "Fo": "Orthic Ferralsols",
    "Fx": "Xanthic Ferralsols",
    "Fr": "Rhodic Ferralsols",
    "Fh": "Humic Ferralsols",
    "Fa": "Acric Ferralsols",
    "Fp": "Plinthic Ferralsols",
    "G": "GLEYSOLS",
    "Ge": "Eutric Gleysols",
    "Gc": "Calcaric Gleysols",
    "Gd": "Dystric Gleysols",
    "Gm": "Mollic Gleysols",
    "Gh": "Humic Gleysols",
    "Gp": "Plinthic Gleysols",
    "Gx": "Gelic Gleysols",
    "H": "PHAEOZEMS",
    "Hh": "Haplic Phaeozems",
    "Hc": "Calcaric Phaeozems",
    "Hl": "Luvic Phaeozems",
    "Hg": "Gleyic Phaeozems",
    "I": "LITHOSOLS",
    "J": "FLUVISOLS",
    "Je": "Eutric Fluvisols",
    "Jc": "Calcaric Fluvisols",
    "Jd": "Dystric Fluvisols",
    "Jt": "Thionic Fluvisols",
    "K": "KASTANOZEMS",
    "Kh": "Haplic Kastanozems",
    "Kk": "Calcic Kastanozems",
    "Kl": "Luvic Kastanozems",
    "L": "LUVISOLS",
    "Lo": "Orthic Luvisols",
    "Lc": "Chromic Luvisols",
    "Lk": "Calcic Luvisols",
    "Lv": "Vertic Luvisols",
    "Lf": "Ferric Luvisols",
    "La": "Albic Luvisols",
    "Lp": "Plinthic Luvisols",
    "Lg": "Gleyic Luvisols",
    "M": "GREYZEMS",
    "Mo": "Orthic Greyzems",
    "Mg": "Gleyic Greyzems",
    "N": "NITOSOLS",
    "Ne": "Eutric Nitosols",
    "Nd": "Dystric Nitosols",
    "Nh": "Humic Nitosols",
    "O": "HISTOSOLS",
    "Oe": "Eutric Histosols",
    "Od": "Dystric Histosols",
    "Ox": "Gelic Histosols",
    "P": "PODZOLS",
    "Po": "Orthic Podzols",
    "Pl": "Leptic Podzols",
    "Pf": "Ferric Podzols",
    "Ph": "Humic Podzols",
    "Pp": "Placic Podzols",
    "Pg": "Gleyic Podzols",
    "Q": "ARENOSOLS",
    "Qc": "Cambic Arenosols",
    "Ql": "Luvic Arenosols",
    "Qf": "Ferralic Arenosols",
    "Qa": "Albic Arenosols",
    "R": "REGOSOLS",
    "Re": "Eutric Gleysols",
    "Rc": "Calcaric Regosols",
    "Rd": "Dystric Regosols",
    "Rx": "Gelic Regosols",
    "S": "SOLONETZ",
    "So": "Orthic Solonetz",
    "Sm": "Mollic Solonetz",
    "Sg": "Gleyic Solonetz",
    "T": "ANDOSOLS",
    "To": "Ochric Andosols",
    "Tm": "Mollic Andosols",
    "Th": "Humic Andosols",
    "Tv": "Vitric Andosols",
    "U": "RANKERS",
    "V": "VERTISOLS",
    "Vp": "Pellic Vertisols",
    "Vc": "Chromic Vertisols",
    "W": "PLANOSOLS",
    "We": "Eutric Planosols",
    "Wd": "Dystric Planosols",
    "Wm": "Mollic Planosols",
    "Wh": "Humic Planosols",
    "Ws": "Solodic Planosols",
    "Wx": "Gelic Planosols",
    "X": "XEROSOLS",
    "Xh": "Haplic Xerosols",
    "Xk": "Calcic Xerosols",
    "Xy": "Gypsic Xerosols",
    "Xl": "Luvic Xerosols",
    "Y": "YERMOSOLS",
    "Yh": "Haplic Yermosols",
    "Yk": "Calcic Yermosols",
    "Yy": "Gypsic Yermosols",
    "Yl": "Luvic Yermosols",
    "Yt": "Takyric Yermosols",
    "Z": "SOLONCHAKS",
    "Zo": "Orthic Solonchaks",
    "Zm": "Mollic Solonchaks",
    "Zt": "Takyric Solonchaks",
    "Zg": "Gleyic Solonchaks",
    "DS": "Dunes or shifting sands",
    "ST": "Salt flats",
    "RK": "Rock debris or desert detritus",
    "WR": "Inland water or ocean",
    "GL": "Glacier",
    "ND": "No data",
}

# create column ssn in map where the value is the value of the key in specific_soil_name that matches the value of DOMSOI in map

map["ssn"] = map["DOMSOI"].map(specific_soil_name)

map

In [20]:
extras = {
    "DS": "Dunes or shifting sands",
    "ST": "Salt flats",
    "RK": "Rock debris or desert detritus",
    "WR": "Inland water or ocean",
    "GL": "Glacier",
    "ND": "No data",
}

# remove rows of map where DOMSOI value is in extras and put in new df

extras_map = map[map["DOMSOI"].isin(extras.keys())]

no_extras_map = map[~map["DOMSOI"].isin(extras.keys())]

In [ ]:
extras_map["dsn"] = map["DOMSOI"].map(extras)

extras_map

In [ ]:
# map no_extras_map dsn value to first letter of value in DOMSOI column that matches key in specific_soil_name

no_extras_map["dsn"] = no_extras_map["DOMSOI"].str[:1].map(specific_soil_name)

no_extras_map

In [ ]:
# concat no_extras_map and extras_map

final = pd.concat([extras_map, no_extras_map])

final

In [ ]:
# print rows of final where dsn is missing
final[final["dsn"].isna()]

In [ ]:
# read in xls file
dmsw_soil_stats = pd.read_excel(
    "/media/muskrat/T7 Shield/eco_data/v3/maps/soil/DSMW/BasicFilesSC.xls"
)

dmsw_soil_stats

In [ ]:
# drop column unnamed: 0

dmsw_soil_stats = dmsw_soil_stats.drop(columns="Unnamed: 0")

dmsw_soil_stats

In [ ]:
# rename columns to values in first row

dmsw_soil_stats.columns = dmsw_soil_stats.iloc[0]

dmsw_soil_stats

In [ ]:
# remove first row

dmsw_soil_stats = dmsw_soil_stats.iloc[1:]

dmsw_soil_stats

In [ ]:
# drop phase code column

dmsw_soil_stats = dmsw_soil_stats.drop(columns="Phase Code")

dmsw_soil_stats

In [34]:
# reset index to start at 0

dmsw_soil_stats.reset_index(drop=True, inplace=True)

In [ ]:
dmsw_soil_stats["soil_texture"] = "unknown"

for i in range(len(dmsw_soil_stats)):
    course = ""
    medium = ""
    fine = ""
    if dmsw_soil_stats["% of MU with Coarse Texture"][i] == 0:
        course = ""
    else:
        course = f"{dmsw_soil_stats['% of MU with Coarse Texture'][i]}% coarse"
    if dmsw_soil_stats["% of MU with Medium Texture"][i] == 0:
        medium = ""
    else:
        medium = f"{dmsw_soil_stats['% of MU with Medium Texture'][i]}% medium"
    if dmsw_soil_stats["% of MU with Heavy Texture"][i] == 0:
        fine = ""
    else:
        fine = f"{dmsw_soil_stats['% of MU with Heavy Texture'][i]}% fine"

    dmsw_soil_stats["soil_texture"][i] = f"{course} {medium} {fine}"

dmsw_soil_stats

In [36]:
dmsw_soil_stats["soil_slope"] = "unknown"

for i in range(len(dmsw_soil_stats)):
    flat = ""
    rolling = ""
    mountainous = ""
    if dmsw_soil_stats["% of MU with Flat Topography"][i] == 0:
        flat = ""
    else:
        flat = f"{dmsw_soil_stats['% of MU with Flat Topography'][i]}% flat"
    if dmsw_soil_stats["% of MU with Rolling Topography"][i] == 0:
        rolling = ""
    else:
        rolling = f"{dmsw_soil_stats['% of MU with Rolling Topography'][i]}% rolling"
    if dmsw_soil_stats["% of MU with Mountaineous Topography"][i] == 0:
        mountainous = ""
    else:
        mountainous = (
            f"{dmsw_soil_stats['% of MU with Mountaineous Topography'][i]}% mountainous"
        )

    dmsw_soil_stats["soil_slope"][i] = f"{flat} {rolling} {mountainous}"

In [ ]:
dmsw_soil_stats

In [ ]:
# drop columns % of MU with Flat Topography, % of MU with Rolling Topography, % of MU with Mountaineous Topography, % of MU with Coarse Texture, % of MU with Medium Texture, % of MU with Heavy Texture

dmsw_soil_stats = dmsw_soil_stats.drop(
    columns=[
        "% of MU with Flat Topography",
        "% of MU with Rolling Topography",
        "% of MU with Mountaineous Topography",
        "% of MU with Coarse Texture",
        "% of MU with Medium Texture",
        "% of MU with Heavy Texture",
    ]
)

dmsw_soil_stats

In [ ]:
# rename column % of Dominant Soil in the MU to dominant_soil_percentage

dmsw_soil_stats = dmsw_soil_stats.rename(
    columns={"% of Dominant Soil in the MU": "dominant_soil_percentage"}
)

dmsw_soil_stats

In [ ]:
# rename dominant_soil_percentage to dominant_soil_type_percentage

dmsw_soil_stats = dmsw_soil_stats.rename(
    columns={"dominant_soil_percentage": "dominant_soil_type_percentage"}
)

# add % to dominant_soil_type_percentage

dmsw_soil_stats["dominant_soil_type_percentage"] = dmsw_soil_stats[
    "dominant_soil_type_percentage"
].astype(str)

dmsw_soil_stats["dominant_soil_type_percentage"] = (
    dmsw_soil_stats["dominant_soil_type_percentage"] + "%"
)

dmsw_soil_stats

In [ ]:
# rename Record Number to SNUM

dmsw_soil_stats = dmsw_soil_stats.rename(columns={"Record Number": "SNUM"})

# merge dmsw_soil_stats with final on SNUM

merged = pd.merge(final, dmsw_soil_stats, on="SNUM", how="outer")

merged

In [ ]:
# drop columns Mapping Unit Name, DOMSOI

merged = merged.drop(columns=["Mapping Unit Name", "DOMSOI"])

merged

In [ ]:
# rename SNUM to soil_id, ssn to specific_soil_name, and dsn to dominant_soil_name

merged = merged.rename(
    columns={
        "SNUM": "soil_id",
        "ssn": "specific_soil_name",
        "dsn": "dominant_soil_name",
    }
)

merged

In [ ]:
# set crs to 4326

merged = merged.set_crs("EPSG:4326")
merged.crs

In [ ]:
# write to geojson

merged.to_file(
    "/media/muskrat/T7 Shield/eco_data/v3/maps/soil/DMSW.geojson", driver="GeoJSON"
)

In [ ]:
# CREATE DB VERSION OF MAP
def get_first_coord_pair(geom):
    if geom.type == "MultiPolygon":
        return geom.geoms[0].exterior.coords[0]
    elif geom.type == "Polygon":
        return geom.exterior.coords[0]
    else:
        raise ValueError("Unsupported geometry type")


map["coordinates"] = map["geometry"].apply(get_first_coord_pair)

map

In [ ]:
# convert map.coordinates to list

map["coordinates"] = map["coordinates"].apply(lambda x: list(x))

map

In [ ]:
# drop geometry column and write map to json file

map.drop(columns=["geometry"], inplace=True)

map

In [ ]:
# convert map from geopandas dataframe to pandas dataframe

map_pd = pd.DataFrame(map)

map

In [8]:
# write map_pd to json file

map_pd.to_json(
    "/media/muskrat/T7 Shield/eco_data/v4/maps/db/DMSW.json",
    orient="records",
    force_ascii=False,
)